In [1]:
import pandas as pd

In [3]:
# Carrega cada arquivo separadamente para análise
file1 = pd.read_parquet("./data/part-00000-tid-2779033056155408584-f6316110-4c9a-4061-ae48-69b77c7c8c36-4-1-c000.snappy.parquet")
file2 = pd.read_parquet("./data/part-00000-tid-5196563791502273604-c90d3a24-52f2-4955-b4ec-fb143aae74d8-4-1-c000.snappy.parquet")  
file3 = pd.read_parquet("./data/part-00000-tid-7173294866425216458-eae53fbf-d19e-4130-ba74-78f96b9675f1-4-1-c000.snappy.parquet")

# Analise cada um
print("Arquivo 1:")
print(f"Shape: {file1.shape}")
print(f"Colunas: {list(file1.columns)}")
print()

print("Arquivo 2:")
print(f"Shape: {file2.shape}") 
print(f"Colunas: {list(file2.columns)}")
print()

print("Arquivo 3:")
print(f"Shape: {file3.shape}")
print(f"Colunas: {list(file3.columns)}")

Arquivo 1:
Shape: (14419, 4)
Colunas: ['pdv', 'premise', 'categoria_pdv', 'zipcode']

Arquivo 2:
Shape: (6560698, 11)
Colunas: ['internal_store_id', 'internal_product_id', 'distributor_id', 'transaction_date', 'reference_date', 'quantity', 'gross_value', 'net_value', 'gross_profit', 'discount', 'taxes']

Arquivo 3:
Shape: (7092, 8)
Colunas: ['produto', 'categoria', 'descricao', 'tipos', 'label', 'subcategoria', 'marca', 'fabricante']


In [4]:
# 1. Cadastro de PDVs
pdvs = file1

# 2. Concatena as transações (são partições do mesmo dataset)
transacoes = pd.concat([file2, file3], ignore_index=True)
print(f"Total de transações: {transacoes.shape[0]:,}")  # ~13.1M registros

# 3. Join das transações com cadastro de PDVs
df_completo = transacoes.merge(
    pdvs, 
    left_on='internal_store_id', 
    right_on='pdv', 
    how='left'
)

Total de transações: 6,567,790


In [5]:
# Verifique se todos os PDVs das transações existem no cadastro
pdvs_transacoes = set(transacoes['internal_store_id'].unique())
pdvs_cadastro = set(pdvs['pdv'].unique())

print(f"PDVs nas transações: {len(pdvs_transacoes):,}")
print(f"PDVs no cadastro: {len(pdvs_cadastro):,}")
print(f"PDVs sem cadastro: {len(pdvs_transacoes - pdvs_cadastro):,}")

PDVs nas transações: 15,087
PDVs no cadastro: 14,419
PDVs sem cadastro: 668


In [6]:
# 1. Identifica quais PDVs não têm cadastro
pdvs_orfaos = transacoes[
    ~transacoes['internal_store_id'].isin(pdvs['pdv'])
]['internal_store_id'].unique()

print(f"PDVs órfãos: {len(pdvs_orfaos)}")

# 2. Calcula o volume de vendas dos PDVs órfãos
vendas_orfaos = transacoes[
    transacoes['internal_store_id'].isin(pdvs_orfaos)
]['quantity'].sum()

# 3. Volume total de vendas
vendas_totais = transacoes['quantity'].sum()

# 4. Percentual
percentual = 100 * vendas_orfaos / vendas_totais

print(f"\n📈 ANÁLISE DE IMPACTO:")
print(f"Vendas em PDVs órfãos: {vendas_orfaos:,}")
print(f"Vendas totais: {vendas_totais:,}") 
print(f"% do volume em PDVs órfãos: {percentual:.2f}%")

# 5. Análise adicional - número de transações
transacoes_orfaos = len(transacoes[
    transacoes['internal_store_id'].isin(pdvs_orfaos)
])
transacoes_totais = len(transacoes)

print(f"% de transações em PDVs órfãos: {100*transacoes_orfaos/transacoes_totais:.2f}%")

PDVs órfãos: 668

📈 ANÁLISE DE IMPACTO:
Vendas em PDVs órfãos: 215,674.66422250867
Vendas totais: 53,311,532.07052398
% do volume em PDVs órfãos: 0.40%
% de transações em PDVs órfãos: 0.80%


In [7]:
df_completo = transacoes.merge(
    pdvs, 
    left_on='internal_store_id', 
    right_on='pdv', 
    how='inner'  # só mantém quem tem match
)
df_completo.head()

,internal_store_id,internal_product_id,distributor_id,transaction_date,reference_date,quantity,gross_value,net_value,gross_profit,discount,...,descricao,tipos,label,subcategoria,marca,fabricante,pdv,premise,categoria_pdv,zipcode
0,7384367747233276219,328903483604537190,9,2022-07-13,2022-07-01,1.0,38.125000,37.890625,10.042625,3.950000,...,NaN,NaN,NaN,NaN,NaN,NaN,7384367747233276219,Off Premise,Package/Liquor,80905
1,3536908514005606262,5418855670645487653,5,2022-03-21,2022-03-01,6.0,107.250000,106.440002,24.732002,17.100000,...,NaN,NaN,NaN,NaN,NaN,NaN,3536908514005606262,Off Premise,Package/Liquor,80239
2,3138231730993449825,1087005562675741887,6,2022-09-06,2022-09-01,3.0,56.625000,56.220001,14.124002,5.250000,...,NaN,NaN,NaN,NaN,NaN,NaN,3138231730993449825,Off Premise,Package/Liquor,80634
3,3681167389484217654,1401422983880045188,5,2022-09-11,2022-09-01,129.0,1037.160023,1037.160023,156.348026,479.880006,...,NaN,NaN,NaN,NaN,NaN,NaN,3681167389484217654,Off Premise,Package/Liquor,80226
4,7762413312337359369,6614994347738381720,4,2022-02-18,2022-02-01,1.0,26.230000,23.950241,6.550241,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,7762413312337359369,Off Premise,Convenience,30096


In [8]:
# Verifica o resultado final
print(f"📊 DATASET CONSOLIDADO:")
print(f"Shape: {df_completo.shape}")
print(f"Período: {df_completo['transaction_date'].min()} até {df_completo['transaction_date'].max()}")
print(f"PDVs únicos: {df_completo['internal_store_id'].nunique():,}")
print(f"Produtos únicos: {df_completo['internal_product_id'].nunique():,}")
print(f"\nMissing values:")
print(df_completo.isnull().sum())

📊 DATASET CONSOLIDADO:
Shape: (6515116, 23)
Período: 2022-01-01 até 2022-12-31
PDVs únicos: 14,419
Produtos únicos: 7,070

Missing values:
internal_store_id            0
internal_product_id          0
distributor_id               0
transaction_date             0
reference_date               0
quantity                     0
gross_value                  0
net_value                    0
gross_profit                 0
discount                     0
taxes                        0
produto                6515116
categoria              6515116
descricao              6515116
tipos                  6515116
label                  6515116
subcategoria           6515116
marca                  6515116
fabricante             6515116
pdv                          0
premise                      0
categoria_pdv                0
zipcode                      0
dtype: int64


In [9]:
# Salva em formato otimizado
df_completo.to_parquet('./data/dataset_consolidado.parquet', index=False)
print("✅ Dataset salvo como 'dataset_consolidado.parquet'")

✅ Dataset salvo como 'dataset_consolidado.parquet'


In [10]:
# Sample de 10% para testes rápidos
sample_df = df_completo.sample(frac=0.1, random_state=42)
sample_df.to_parquet('./data/sample_dataset.parquet', index=False)
print("✅ Sample 10% salvo como 'sample_dataset.parquet'")

✅ Sample 10% salvo como 'sample_dataset.parquet'
